### Prepare the dataset
1 line per document

In [1]:
from pathlib import Path
import os

import pandas as pd

In [3]:
DirPpath = Path(os.path.abspath('')).parent
file = str(DirPpath) + "\DataCollection\Files\corpus.csv"

CorpusDF = pd.read_csv(file)

In [5]:
CorpusDF.isna().sum().sum()

0

In [6]:
CorpusDF = CorpusDF.dropna()

CorpusDF['0'] = CorpusDF['0'].apply(lambda x: x.replace('skip to main content this site uses cookies to offer you a better browsing experience. find out more on how we use cookies. accept all cookies accept only essential cookies an official website of the european union', ''))

In [5]:
file = str(DirPpath) + "\Fine-tuning\Files\corpus.csv"
CorpusDF.to_csv(file)

In [6]:
file = str(DirPpath) + "\Fine-tuning\Files\corpus.txt"
with open(file, 'w', encoding='utf-8') as f:
    for ID, content in zip(CorpusDF['Unnamed: 0'].values, CorpusDF['0'].values):
        f.write('\n'.join([str(ID), content]))

### Extract vocabulary: 

In [35]:
from tokenizers import BertWordPieceTokenizer
from transformers import RobertaTokenizer
import transformers
import tokenizers

In [8]:
# Essentials
# LOCAL_INPUT_PATH is mapped to S3 input location for covid news articles 
LOCAL_INPUT_PATH = file 
# LOCAL_OUTPUT_PATH is mapped to S3 output location where we want to save the custom vocabulary after training the tokenizer
LOCAL_OUTPUT_PATH =  str(DirPpath) + "vocab"
VOCAB_SIZE = 30522

In [9]:
paths = file

tokenizer = BertWordPieceTokenizer()
tokenizer.train(files=paths, vocab_size=VOCAB_SIZE)


In [11]:
DirPpath = Path(os.path.abspath('')).parent
OutputPath = str(DirPpath) + "\Fine-tuning\Files"

tokenizer.save_model(OutputPath)


['c:\\Users\\ecaudron001\\Documents\\GitHub\\LLM-for-Tourism\\Fine-tuning\\Files\\vocab.txt']

In [37]:
tokenizer = BertWordPieceTokenizer('vocab.txt')

# Evaluate custom tokenizer 
print('Evaluating custom tokenizer')
test_sentence = 'covid is a virus'
print(f'Test sentence: {test_sentence}')
tokens = tokenizer.encode(test_sentence).tokens
print(f'Encoded sentence: {tokens}')
token_id = tokenizer.token_to_id('covid')
print(f'Token ID for token (covid) = {token_id}')
vocab_size = tokenizer.get_vocab_size()
print(f'Vocabulary size = {vocab_size}')

Evaluating custom tokenizer
Test sentence: covid is a virus
Encoded sentence: ['[CLS]', 'covid', 'is', 'a', 'virus', '[SEP]']
Token ID for token (covid) = 976
Vocabulary size = 27799


In [ ]:
df = pd.read_csv('vocab.txt', delimiter="/t", header=None)

In [ ]:
df.to_csv("vocab.csv")

Roberta

In [4]:
from tokenizers import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
from pathlib import Path

paths = str(DirPpath) + "\Fine-tuning\Files\corpus.txt"

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer(lowercase=True)

# Customize training
tokenizer.train(files=paths, vocab_size=8192, min_frequency=2,
                show_progress=True,
                special_tokens=[
                                "<s>",
                                "<pad>",
                                "</s>",
                                "<unk>",
                                "<mask>",
])
#Save the Tokenizer to disk
output = str(DirPpath) + "\Fine-tuning"
tokenizer.save_model(output)

['c:\\Users\\ecaudron001\\Documents\\GitHub\\LLM-for-Tourism\\Fine-tuning\\vocab.json',
 'c:\\Users\\ecaudron001\\Documents\\GitHub\\LLM-for-Tourism\\Fine-tuning\\merges.txt']

In [5]:
df = pd.read_csv('merges.txt', delimiter="/t", header=None)
df.to_csv("merges.csv")

C:\Users\ecaudron001\AppData\Local\Temp\ipykernel_16236\3076094962.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('merges.txt', delimiter="/t", header=None)
